Importation des packages nécessaire pour extraire des données.

In [102]:
# Importation des packages
import requests
from bs4 import BeautifulSoup
import csv
from urllib.parse import urlencode
import pandas as pd
import json

Création d'une fonction qui permet de récupéter l'URL toutes les recettes du site marmiton.org avec un ingrédient. Ici nous prenons l'exemple de l'ingrédient "concombre". Le site restreint le nombre de recettes à un maximum de 1 000.

In [103]:
def get_recipes_url_whit_ingredient(ingredient):
    all_recipes_url = []
    base_url = "https://www.marmiton.org/recettes/recherche.aspx"
    page_number = 1
    
    while True:
        search_params = {"aqt": ingredient, "page": page_number}
        search_url = f"{base_url}?{urlencode(search_params)}"
        response = requests.get(search_url)
        
        if response.status_code == 404:
            break
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        link_recipes = soup.find_all("a", class_="MRTN__sc-1gofnyi-2 gACiYG")
        
    
        for link_recipe in link_recipes:
            url_recipe = "https://www.marmiton.org" + link_recipe.get('href')
            all_recipes_url.append(url_recipe)
            
        page_number += 1
            
    pd.set_option('display.max_colwidth', 1000)
    df_all_recipes_url = pd.DataFrame(all_recipes_url , columns=["Recipe URL"])
    
    
    return(df_all_recipes_url)

recipes_with_concombre = get_recipes_url_whit_ingredient("concombre")
print(recipes_with_concombre)

                                                                                               Recipe URL
0    https://www.marmiton.org/recettes/recette_verrines-au-saumon-fume-a-la-creme-de-concombre_55922.aspx
1                                 https://www.marmiton.org/recettes/recette_soupe-de-concombre_15059.aspx
2           https://www.marmiton.org/recettes/recette_tagliatelles-de-concombre-au-saumon-fume_64364.aspx
3                        https://www.marmiton.org/recettes/recette_tzatziki-de-concombre-grece_12823.aspx
4         https://www.marmiton.org/recettes/recette_salade-fraicheur-concombre-et-surimi-light_34120.aspx
..                                                                                                    ...
995                           https://www.marmiton.org/recettes/recette_caviar-et-gourmandise_227907.aspx
996                               https://www.marmiton.org/recettes/recette_verrine-asiatique_224851.aspx
997                        https://www.marmito

Création d'une fonction qui permet d'extraire le nom de la recette.

In [104]:
def recipe_title(url):
    recipe_url = url
    try:
        response = requests.get(recipe_url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        # En cas d'erreur de connexion, on passe à la page suivante
        print(f"Erreur de connexion à {recipe_url}. La page suivante sera traitée.")
        return "N/A"
    
    soup = BeautifulSoup(response.content, "html.parser")
    div_recipe_title = soup.find_all("div", class_="RCP__sc-l87aur-2 PLTXZ")
    
    if not div_recipe_title:
        text_recipe_titles = "N/A"
    else:
        text_recipe_titles = [recipe.find("h1").text.strip() for recipe in div_recipe_title]
    
    return text_recipe_titles

On applique notre fonction dans dataframe pour récupérer tous les noms des recettes utilisant comme ingrédient "concombre"

In [105]:
recipes_with_concombre["Recipe Title"] = recipes_with_concombre["Recipe URL"].apply(recipe_title)
print(recipes_with_concombre)

Erreur de connexion à https://www.marmiton.orghttps://www.marmiton.org/recettes/album1379858/5-recettes-fraiches-avec-du-concombre-0.html. La page suivante sera traitée.
Erreur de connexion à https://www.marmiton.orghttps://www.marmiton.org/recettes/album1379841/concombre-10-recettes-qui-changent-vraiment-0.html. La page suivante sera traitée.
Erreur de connexion à https://www.marmiton.orghttps://www.marmiton.org/recettes/album1378413/pas-a-pas-la-salade-de-concombres-a-la-creme-fraiche-0.html. La page suivante sera traitée.
Erreur de connexion à https://www.marmiton.orghttps://www.marmiton.org/recettes/album1355260/20-recettes-pour-profiter-du-concombre-0.html. La page suivante sera traitée.
Erreur de connexion à https://www.marmiton.org/recettes/index/ingredient/1-concombre. La page suivante sera traitée.
                                                                                               Recipe URL  \
0    https://www.marmiton.org/recettes/recette_verrines-au-saumon-fume-a

On supprime de notra DataFrame, les valeurs manquantes

In [106]:
recipes_with_concombre.drop(recipes_with_concombre[recipes_with_concombre['Recipe Title'] == 'N/A'].index, inplace=True)
recipes_with_concombre.dropna(how='all', inplace=True)

Développement d'une fonction qui, en utilisant l'URL d'une recette, extrait la liste des ingrédients de cette recette ainsi que le nombre de personnes associé.

In [107]:
def list_ingredients(url):
    recipe_url = url
    try:
        response = requests.get(recipe_url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        # En cas d'erreur de connexion, on passe à la page suivante
        print(f"Erreur de connexion à {recipe_url}. La page suivante sera traitée.")
        return "N/A"
    soup = BeautifulSoup(response.content, "html.parser")
    script_tag = soup.find("script", type="application/ld+json")
    
    if script_tag:
        # Extraire le contenu JSON à l'intérieur de la balise <script>
        json_data = script_tag.string.strip()

        # Charger les données JSON
        data = json.loads(json_data)

        # Extraire les valeurs spécifiques
        recipe_yield = data.get("recipeYield", "N/A")
        recipe_ingredients = data.get("recipeIngredient", [])

        # Créer un dictionnaire de données pour le DataFrame
        recipe_data = {
            "nbr de personnes": [recipe_yield]
        }

        # Ajouter chaque ingrédient en tant que colonne dans le dictionnaire
        for i, ingredient in enumerate(recipe_ingredients, start=1):
            recipe_data[f"ingredient {i}"] = [ingredient]
            
            

        # Créer le DataFrame
        df_ingredients = pd.DataFrame(recipe_data)

  
    else:
        print("Balise <script> non trouvée ou type incorrect.")
        df_ingredients = None
    
    return df_ingredients

Application de la fonction dans notre DataFrame

In [108]:
intermediate_dfs = recipes_with_concombre.apply(lambda row: list_ingredients(row["Recipe URL"]), axis=1)

In [110]:
result_df = pd.concat(intermediate_dfs.tolist(), ignore_index=True)

In [111]:
recipes_with_concombre_ingredients = pd.concat([recipes_with_concombre, result_df], axis=1)

In [112]:
recipes_with_concombre_ingredients.drop(recipes_with_concombre[recipes_with_concombre['Recipe Title'] == 'N/A'].index, inplace=True)
recipes_with_concombre_ingredients.dropna(how='all', inplace=True)

In [115]:
def list_steps(url):
    recipe_url = url
    try:
        response = requests.get(recipe_url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        # En cas d'erreur de connexion, retourner un DataFrame vide
        return pd.DataFrame(columns=["nbre d'étapes", "Etape 1"])
    
    soup = BeautifulSoup(response.content, "html.parser")
    script_tag = soup.find("script", type="application/ld+json")
    
    if script_tag:
        # Extraire le contenu JSON à l'intérieur de la balise <script>
        json_data = script_tag.string.strip()

        # Charger les données JSON
        data = json.loads(json_data)

        #Extrare les données dans recipeinstructions
        recipe_instructions = data.get("recipeInstructions", [])
        
        # Compter le nombre d'étapes de type "HowToStep"
        num_steps = sum(1 for step in recipe_instructions if step.get("@type") == "HowToStep")
        
        # Créer un dictionnaire de données pour le DataFrame
        recipe_data = {
            "nbre d'étapes": [num_steps]
            }
        

        # Ajouter chaque étape en tant que colonne dans le dictionnaire
        for i, instruction in enumerate(recipe_instructions, start=1):
            recipe_data[f"Etape {i}"] = instruction.get("text", "")
            
            


        # Créer le DataFrame
        df_steps = pd.DataFrame(recipe_data)

  
    else:
        print("Balise <script> non trouvée ou type incorrect.")
        df_steps = None
    
    return df_steps

In [119]:
intermediate_dfs_1 = recipes_with_concombre_ingredients.apply(lambda row: list_steps(row["Recipe URL"]), axis=1)
result_df_1 = pd.concat(intermediate_dfs_1.tolist(), ignore_index=True)
recipes_with_concombre_ingredients_steps = pd.concat([recipes_with_concombre_ingredients, result_df_1], axis=1)

In [120]:
# Définir le nom du fichier Excel de sortie
output_excel_file = "concombre_data.xlsx"
recipes_with_concombre_ingredients_steps.to_excel(output_excel_file, index=False)

In [121]:
#Applications avec l'ingredient tomate
recipes_with_tomate = get_recipes_url_whit_ingredient("tomate")
recipes_with_tomate["Recipe Title"] = recipes_with_tomate["Recipe URL"].apply(recipe_title)
recipes_with_tomate.drop(recipes_with_tomate[recipes_with_tomate['Recipe Title'] == 'N/A'].index, inplace=True)
recipes_with_tomate.dropna(how='all', inplace=True)
intermediate_dfs = recipes_with_tomate.apply(lambda row: list_ingredients(row["Recipe URL"]), axis=1)
result_df = pd.concat(intermediate_dfs.tolist(), ignore_index=True)
recipes_with_tomate_ingredients = pd.concat([recipes_with_tomate, result_df], axis=1)
recipes_with_tomate_ingredients.drop(recipes_with_tomate_ingredients[recipes_with_tomate_ingredients['Recipe Title'] == 'N/A'].index, inplace=True)
recipes_with_tomate_ingredients.dropna(how='all', inplace=True)
intermediate_dfs_1 = recipes_with_tomate_ingredients.apply(lambda row: list_steps(row["Recipe URL"]), axis=1)
result_df_1 = pd.concat(intermediate_dfs_1.tolist(), ignore_index=True)
recipes_with_tomate_ingredients_steps = pd.concat([recipes_with_tomate_ingredients, result_df_1], axis=1)
output_excel_file = "tomate_data.xlsx"
recipes_with_tomate_ingredients_steps.to_excel(output_excel_file, index=False)







In [135]:
ingredient = "banane"
output_excel_file = "banane_data.xlsx"

# Etape 1 : Obtenir les URL de recettes avec l'ingrédient donné
recipes_with_ingredient = get_recipes_url_whit_ingredient(ingredient)
    
# Etape 2 : Obtenir le titre des recettes
recipes_with_ingredient["Recipe Title"] = recipes_with_ingredient["Recipe URL"].apply(recipe_title)
    
# Etape 3 : Supprimer les lignes avec des titres de recettes "N/A" et les lignes vides
recipes_with_ingredient.drop(recipes_with_ingredient[recipes_with_ingredient['Recipe Title'] == 'N/A'].index, inplace=True)
recipes_with_ingredient.dropna(how='all', inplace=True)
    
# Etape 4 : Obtenir les ingrédients pour chaque recette
intermediate_dfs = recipes_with_ingredient.apply(lambda row: list_ingredients(row["Recipe URL"]), axis=1)
result_df = pd.concat(intermediate_dfs.tolist(), ignore_index=True)
recipes_with_ingredient_ingredients = pd.concat([recipes_with_ingredient, result_df], axis=1)
    
# Etape 5 : Supprimer les lignes avec des titres de recettes "N/A" et les lignes vides
recipes_with_ingredient_ingredients.drop(recipes_with_ingredient_ingredients[recipes_with_ingredient_ingredients['Recipe Title'] == 'N/A'].index, inplace=True)
recipes_with_ingredient_ingredients.dropna(how='all', inplace=True)
    
# Etape 6 : Obtenir les étapes pour chaque recette
intermediate_dfs_1 = recipes_with_ingredient_ingredients.apply(lambda row: list_steps(row["Recipe URL"]), axis=1)
result_df_1 = pd.concat(intermediate_dfs_1.tolist(), ignore_index=True)
recipes_with_ingredient_ingredients_steps = pd.concat([recipes_with_ingredient_ingredients, result_df_1], axis=1)
    
# Etape 7 : Exporter les données finales au format Excel
recipes_with_ingredient_ingredients_steps.to_excel(output_excel_file, index=False)
print(f"Les données pour l'ingrédient '{ingredient}' ont été exportées au fichier Excel : {output_excel_file}")
    
    
    

Les données pour l'ingrédient 'banane' ont été exportées au fichier Excel : banane_data.xlsx


In [136]:
ingredient = "abricot"
output_excel_file = "abricot_data.xlsx"

# Etape 1 : Obtenir les URL de recettes avec l'ingrédient donné
recipes_with_ingredient = get_recipes_url_whit_ingredient(ingredient)
    
# Etape 2 : Obtenir le titre des recettes
recipes_with_ingredient["Recipe Title"] = recipes_with_ingredient["Recipe URL"].apply(recipe_title)
    
# Etape 3 : Supprimer les lignes avec des titres de recettes "N/A" et les lignes vides
recipes_with_ingredient.drop(recipes_with_ingredient[recipes_with_ingredient['Recipe Title'] == 'N/A'].index, inplace=True)
recipes_with_ingredient.dropna(how='all', inplace=True)
    
# Etape 4 : Obtenir les ingrédients pour chaque recette
intermediate_dfs = recipes_with_ingredient.apply(lambda row: list_ingredients(row["Recipe URL"]), axis=1)
result_df = pd.concat(intermediate_dfs.tolist(), ignore_index=True)
recipes_with_ingredient_ingredients = pd.concat([recipes_with_ingredient, result_df], axis=1)
    
# Etape 5 : Supprimer les lignes avec des titres de recettes "N/A" et les lignes vides
recipes_with_ingredient_ingredients.drop(recipes_with_ingredient_ingredients[recipes_with_ingredient_ingredients['Recipe Title'] == 'N/A'].index, inplace=True)
recipes_with_ingredient_ingredients.dropna(how='all', inplace=True)
    
# Etape 6 : Obtenir les étapes pour chaque recette
intermediate_dfs_1 = recipes_with_ingredient_ingredients.apply(lambda row: list_steps(row["Recipe URL"]), axis=1)
result_df_1 = pd.concat(intermediate_dfs_1.tolist(), ignore_index=True)
recipes_with_ingredient_ingredients_steps = pd.concat([recipes_with_ingredient_ingredients, result_df_1], axis=1)
    
# Etape 7 : Exporter les données finales au format Excel
recipes_with_ingredient_ingredients_steps.to_excel(output_excel_file, index=False)
print(f"Les données pour l'ingrédient '{ingredient}' ont été exportées au fichier Excel : {output_excel_file}")
    

Erreur de connexion à https://www.marmiton.orghttps://www.marmiton.org/recettes/album1369299/20-recettes-a-faire-avec-des-abricots-0.html. La page suivante sera traitée.
Les données pour l'ingrédient 'abricot' ont été exportées au fichier Excel : abricot_data.xlsx
